# step 1

In [37]:
#import Necessary Libraries
import nltk
import pandas as pd
import re

!pip install camel-tools
!camel_data full
!pip install -U farasapy


from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer as stemmer

# Import the dediac_ar function from camel_tools to remove diacritics (Tashkeel)
from camel_tools.utils.dediac import dediac_ar

# Download NLTK stopwords
nltk.download('stopwords')

# Import the stopwords module
from nltk.corpus import stopwords

Usage:
    camel_data (-i | --install) [-f | --force] <PACKAGE>
    camel_data (-p | --post-install) <PACKAGE> <ARGS>...
    camel_data (-l | --list)
    camel_data (-u | --update)
    camel_data (-v | --version)
    camel_data (-h | --help)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# step 2

In [38]:
# Load the dataset
df = pd.read_csv('/content/ArSarcasm_train.csv')
# Inspect the dataset
print("Original Dataset:")
print(df.head()) #shows the fisrt 5 rows
print("Dataset Info:")
print(df.info()) #shows the data information

Original Dataset:
  dialect  sarcasm sentiment original_sentiment  \
0    gulf    False  negative           negative   
1     msa    False   neutral           positive   
2   egypt    False   neutral            neutral   
3  levant     True   neutral           negative   
4     msa    False   neutral           negative   

                                               tweet   source  
0  "نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...  semeval  
1  "#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...  semeval  
2                      "@Alito_NBA اتوقع انه بيستمر"  semeval  
3     "@KSA24 يعني "بموافقتنا" لأن دمشق صايرة موسكو"  semeval  
4  "RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...  semeval  
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8437 entries, 0 to 8436
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8437 non-null   object
 1   sarcasm             84

# step 3

In [39]:

def normalize_text(text):
    # Replace underscore with a space
    text = text.replace('_', ' ')
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove Tatweel
    text = text.replace('ـ', '')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#  Apply normalization to the 'tweet' column
df['tweet'] = df['tweet'].apply(normalize_text)
print("After Normalization:")
print(df['tweet'].head())

After Normalization:
0    نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ك...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد في الحرس يعترف بفقدان السيطرة الأمنية في ...
Name: tweet, dtype: object


# step 4

In [40]:

# Define a function to remove diacritics from Arabic text
def remove_diacritics(text):
 # Use dediac_ar to strip diacritics from the input text
    return dediac_ar(text)

# Apply the diacritic removal function to the 'tweet' column in the dataset
df['tweet'] = df['tweet'].apply(remove_diacritics)

# Print the first few rows of the 'tweet' column after removing diacritics
print("After Removing Diacritics:")
print(df['tweet'].head())

After Removing Diacritics:
0    نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ك...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد في الحرس يعترف بفقدان السيطرة الأمنية في ...
Name: tweet, dtype: object


# step 6

In [41]:


# Load Arabic stopwords from NLTK
arabic_stopwords = set(stopwords.words('arabic'))

# Define the stopword removal function
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word not in arabic_stopwords]  # Filter stopwords
    return ' '.join(filtered_words)  # Rejoin the filtered words into a string

# Apply the function to the 'tweet' column
df['tweet'] = df['tweet'].apply(remove_stopwords)

# Display the cleaned text
print("After Removing Stopwords:")
print(df['tweet'].head())

After Removing Stopwords:
0    نصيحه عمرك اتنزل لعبة سوبر ماريو مش زي كنا متو...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد الحرس يعترف بفقدان السيطرة الأمنية شرقي و...
Name: tweet, dtype: object


# test dataset

# step 2

In [42]:
# Load the dataset

df1 = pd.read_csv('/content/ArSarcasm_test.csv')


In [43]:
# Inspect the dataset
print("Original Dataset:")
print(df1.head()) #shows the fisrt 5 rows
print("Dataset Info:")
print(df1.info()) #shows the data information

Original Dataset:
  dialect  sarcasm sentiment original_sentiment  \
0     msa     True  negative           negative   
1    gulf    False  positive            neutral   
2     msa     True   neutral            neutral   
3     msa    False   neutral            neutral   
4     msa    False   neutral            neutral   

                                               tweet   source  
0  "@AbuEmad74241481 @Cesars2014 هههههه حزب الله ...  semeval  
1  "RT @JannetForster: البنات اللي م صامو بقولكم ...  semeval  
2             اشارة رابعة اشبه بنار تحرق الانقلابيين     astd  
3  "@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ...  semeval  
4  ما لا تراه على التلفاز منافسة شديدة بين المرشح...  semeval  
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             2110 non-null   object
 1   sarcasm             21

# step 3

In [44]:

def normalize_text(text):
    # Replace underscore with a space
    text = text.replace('_', ' ')
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove Tatweel
    text = text.replace('ـ', '')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply normalization to the 'tweet' column
df1['tweet'] = df1['tweet'].apply(normalize_text)
print("After Normalization:")
print(df1['tweet'].head())

After Normalization:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي م صامو بقولكم ترا رمضان قرب و الوق...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علماً اني في بروكسيل و ا...
4    ما لا تراه على التلفاز منافسة شديدة بين المرشح...
Name: tweet, dtype: object


# step 4

In [45]:

# Define a function to remove diacritics from Arabic text
def remove_diacritics(text):
 # Use dediac_ar to strip diacritics from the input text
    return dediac_ar(text)

# Apply the diacritic removal function to the 'tweet' column in the dataset
df1['tweet'] = df1['tweet'].apply(remove_diacritics)

# Print the first few rows of the 'tweet' column after removing diacritics
print("After Removing Diacritics:")
print(df1['tweet'].head())

After Removing Diacritics:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي م صامو بقولكم ترا رمضان قرب و الوق...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علما اني في بروكسيل و ار...
4    ما لا تراه على التلفاز منافسة شديدة بين المرشح...
Name: tweet, dtype: object


# step 6

In [46]:

# Load Arabic stopwords from NLTK
arabic_stopwords = set(stopwords.words('arabic'))

# Define the stopword removal function
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word not in arabic_stopwords]  # Filter stopwords
    return ' '.join(filtered_words)  # Rejoin the filtered words into a string

# Apply the function to the 'tweet' column
df1['tweet'] = df1['tweet'].apply(remove_stopwords)

# Display the cleaned text
print("After Removing Stopwords:")
print(df1['tweet'].head())

After Removing Stopwords:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي صامو بقولكم ترا رمضان قرب الوقت قل...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علما اني بروكسيل اريد ال...
4    تراه التلفاز منافسة شديدة المرشحين هيلاري كلنت...
Name: tweet, dtype: object
